In [105]:
# !pip install webdriver_manager
# !pip install pandas
# !pip install selenium
# !pip install pillow

In [1]:
# import
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import requests
import csv
import os
from PIL import Image
from io import BytesIO

In [4]:
### 찐최종버전 !!!

url = 'https://www.cmcsungmo.or.kr/page/department/A'

all_doctor_info_list = []

# 1. chrome driver 띄우기

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--ssl-version-max=tls1.2")
chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)
driver.get(url)

doctor_info_list = []
    
for i in range(0, 39):
    
    doctor_info_list = []

    # 2. 과 상세 페이지 접근

    mouse = driver.find_elements(By.CLASS_NAME, 'front')
    button = driver.find_elements(By.CLASS_NAME, 'btn-bak1')

    act = ActionChains(driver)
    act.move_to_element(mouse[i]).click(button[i]).perform()

    # 2. 의료진 정보 가져오기
    
    doctor_tags = driver.find_elements(By.CSS_SELECTOR, 'div.doc_list_wrap')

    for j in range(len(doctor_tags)):
        # clinic_elem = "format"
        try:
            if doctor_tags[j]:
                dptm_elem = driver.find_element(By.TAG_NAME, 'h2').text
                name_elem = doctor_tags[j].find_element(By.CSS_SELECTOR, 'strong').text
                position_elem = doctor_tags[j].find_element(By.CSS_SELECTOR, '.doc_img_btn img').get_attribute("alt")
                img_elem = doctor_tags[j].find_element(By.CSS_SELECTOR, '.doc_img_btn img').get_attribute("src")
                
                try:
                    # 3.3. 진료분야 클릭
                    clinic_button = doctor_tags[j].find_element(By.CLASS_NAME, 'reduce_line')
                    clinic_button.click()
                    clinic_elem = doctor_tags[j].find_element(By.CSS_SELECTOR, 'div.reduce_line').text
                except:
                    clinic_elem = "none"
                
                dptm = dptm_elem
                name = name_elem
                _, position, _ = position_elem.split(' ')
                clinic = clinic_elem
                img_src = "https://www.cmcsungmo.or.kr" + img_elem
                img_name = "가톨릭대학교" + " " + "여의도성모병원" + " " + dptm + " " + name + ".png"
                
        except Exception as e:
            print(f"An error occurred: {e}")
            continue
    
        # 3.4. 의료진 데이터 저장
        doctor_info = {
            'dptm': dptm,
            'name': name,
            'img_name': img_name, 
            'position': position,
            'clinic': clinic,
        }
        doctor_info_list.append(doctor_info)
          
    all_doctor_info_list.extend(doctor_info_list)

    # 4.1. 진료과/의료진 버튼 클릭
    dptm_button = driver.find_element(By.CLASS_NAME, 'd_btn_1')
    dptm_button.click()

    # 4.2. 진료과 버튼 클릭
    dptm_button = driver.find_element(By.CSS_SELECTOR, 'a[href="/page/department/A"]')
    dptm_button.click()             
                    
driver.quit()

# 4. csv로 내보내기
with open("04/doctor_info.csv", 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['dptm', 'name', 'img_name', 'position', 'clinic']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for doctor_info in all_doctor_info_list:
        writer.writerow(doctor_info)

An error occurred: too many values to unpack (expected 3)


In [5]:
# 가톨릭대학교 여의도성모병원 url
url = 'https://www.cmcsungmo.or.kr/page/department/A'

# departments =[
#     "종양내과", "진단검사의학과",
#     "치과", "피부과", "핵의학과", "혈관이식외과", "혈액내과", "호스피스완화의학",
#     "호흡기내과", "흉부외과"
# ]
departments = ["호스피스완화의학", "흉부외과"]

all_doctor_info_list = []

# 1. chrome driver 띄우기

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--ssl-version-max=tls1.2")
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)
driver.get(url)
    
for department in departments:
    doctor_info_list = []
    
    # 1. 과 검색

    # 1.1. 검색 버튼 클릭
    search_button = driver.find_element(By.CLASS_NAME, 'btn_search')
    search_button.click()
    
    # 1.2. 검색창 버튼 클릭
    search_box = driver.find_element(By.ID, 'kwd')
    time.sleep(3)
    search_box.click()
    
    # 1.3. 진료과 이름 입력
    search_box.clear() 
    search_box.send_keys(department)
    
    # 1.4. 검색 버튼 클릭
    search_button2 = driver.find_element(By.CLASS_NAME, 'search_btn')
    search_button2.click()

    # 1.5. 의료진 버튼 클릭
    docker_link = driver.find_element(By.XPATH, '//a[contains(@href, "javascript:goCategory(\'DOCTOR\');")]')
    docker_link.click() 

    # 2. 의료진 정보 가져오기

    # 2.1. html 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    soup

    # 3.2. 의료진 정보 가져오기

    doctor_tags = soup.find_all('div', class_='doc_list_wrap')
    clinic_tags = soup.find_all('dt', string='전문분야')

    for i in range(1, len(doctor_tags)):        
        if doctor_tags[i]:
            dptm_name_elem = doctor_tags[i].find('strong').text
            name_elem, dptm_elem = dptm_name_elem.split('[')
            img_elem = doctor_tags[i].find('img')['src']
            position_elem = doctor_tags[i].find('img')['alt']
            clinic_elem = clinic_tags[i].find_next('p').text
            
            if dptm_name_elem and name_elem and dptm_elem and img_elem and position_elem and clinic_elem:
                
                dptm = dptm_elem.strip('[]')
                name = name_elem.strip()
                img_name = "가톨릭대학교" + " " + "여의도성모병원" + " " + department + " " + name + ".png"
                position = position_elem.replace(name, '').split(' ')[0]
                clinic = clinic_elem
                img_src = "https://www.cmcsungmo.or.kr/" + img_elem

                # # 3.3. 이미지 다운로드
                # response = requests.get(img_src, verify=False)
                # if response.status_code == 200:
                #     image_data = response.content
                #     image = Image.open(BytesIO(image_data))
                #     image.save(f"03/images/{img_name}")  

                # 3.4. 의료진 데이터 저장
                doctor_info = {
                    'dptm': dptm,
                    'name': name,
                    'img_name': img_name, 
                    'position': position,
                    'clinic': clinic,
                }
                doctor_info_list.append(doctor_info)
                
        all_doctor_info_list.extend(doctor_info_list)
    
    # 4.1. 진료과/의료진 버튼 클릭
    dptm_button = driver.find_element(By.CLASS_NAME, 'd_btn_1')
    dptm_button.click()
    
    # 4.2. 진료과 버튼 클릭
    dptm_button = driver.find_element(By.CLASS_NAME, 'active_line')
    dptm_button.click()
                        
driver.quit()

# 4. csv로 내보내기
with open("04/doctor_info.csv", 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['dptm', 'name', 'img_name', 'position', 'clinic']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for doctor_info in all_doctor_info_list:
        writer.writerow(doctor_info)

In [ ]:
# 가톨릭대학교 여의도성모병원 url
url = 'https://www.cmcsungmo.or.kr/page/department/A'

all_doctor_info_list = []

# 1. chrome driver 띄우기

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--ssl-version-max=tls1.2")
chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)
driver.get(url)

mouse = driver.find_elements(By.CLASS_NAME, 'front')
button = driver.find_elements(By.CLASS_NAME, 'btn-bak1')
    
# for i in range(len(mouse))
for i in range(1, 4):
    
    doctor_info_list = []
    
    # 1. 과 검색

    act = ActionChains(driver)
    act.move_to_element(mouse[i]).click(button[i]).perform()

    # 2. 의료진 정보 가져오기

    # 2.1. html 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    soup

    # 2.2. 의료진 정보 가져오기

    dptm = soup.find('h2').text
    
    doctor_tags = soup.find_all('div', class_='doc_list_wrap')
    clinic_tags = soup.find_all('p', class_='reduce_line')

    for j in range(1, len(doctor_tags)):        
        if doctor_tags[j]:
            img_elem = doctor_tags[i].find('img')
            alt_text = img_elem['alt']
            img_elem = 
            position_elem = 
            clinic_elem = 
            
            if dptm_name_elem and name_elem and dptm_elem and img_elem and position_elem and clinic_elem:
                
                dptm = dptm_elem.strip('[]')
                name = name_elem.strip()
                img_name = "가톨릭대학교" + " " + "여의도성모병원" + " " + department + " " + name + ".png"
                position = position_elem.replace(name, '').split(' ')[0]
                clinic = clinic_elem
                img_src = "https://www.cmcsungmo.or.kr/" + img_elem

                # # 3.3. 이미지 다운로드
                # response = requests.get(img_src, verify=False)
                # if response.status_code == 200:
                #     image_data = response.content
                #     image = Image.open(BytesIO(image_data))
                #     image.save(f"03/images/{img_name}")  

                # 3.4. 의료진 데이터 저장
                doctor_info = {
                    'dptm': dptm,
                    'name': name,
                    'img_name': img_name, 
                    'position': position,
                    'clinic': clinic,
                }
                doctor_info_list.append(doctor_info)
                
        all_doctor_info_list.extend(doctor_info_list)
    
    # 4.1. 진료과/의료진 버튼 클릭
    dptm_button = driver.find_element(By.CLASS_NAME, 'd_btn_1')
    dptm_button.click()
    
    # 4.2. 진료과 버튼 클릭
    dptm_button = driver.find_element(By.CLASS_NAME, 'active_line')
    dptm_button.click()
                        
driver.quit()

# 4. csv로 내보내기
with open("data/04/doctor_info.csv", 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['dptm', 'name', 'img_name', 'position', 'clinic']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for doctor_info in all_doctor_info_list:
        writer.writerow(doctor_info)